In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import folium
import random
import collections
import unicodedata
import datetime

In [11]:
json_dir = 'Geojson_output_files/'
geo_json_path = r'data/positions-geographiques-des-stations-du-reseau-ratp.geojson'
geo_data_path = r'Database/paris_metropolitan_evolution_db.geojson'
geo_data_line = r'Database/paris_metropolitan_an_evolution_lines.geojson'

In [12]:
paris_geo_1908 = json.load(open(json_dir+'1908_Metropolitan_Stations.geojson'))
paris_geo_1915 = json.load(open(json_dir+'1915_Metro_Paris_stations.geojson'))
paris_geo_1950 = json.load(open(json_dir+'1950_Metro_stations.geojson'))
geo_json_ratp = json.load(open(geo_json_path))
geo_database = json.load(open(geo_data_path))
geo_lines = json.load(open(geo_data_line))

In [572]:
search_station(geo_json_ratp,"Guy-Môquet")

stop_name : Guy-Môquet  |stop_id : 1888
stop_name : Guy-Môquet  |stop_id : 2284


In [65]:
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

def search_station(geo,station):
    for feat in geo: #['features']:
        if strip_accents(station).lower() in strip_accents(feat['properties']['stop_name']).lower():
                print('stop_name :', feat['properties']['stop_name'], 
                      ' |stop_id :',feat['properties']['stop_id'])
                
def search_station_df(df,station):
    catch = []
    for name in df.stop_name.values:
        if strip_accents(station.lower()) in strip_accents(name.lower()):
            catch.append(name)
    print(catch)
    return catch
                
def search_empty(geo):
    for feat in geo:
        if feat['properties']['open_date'] == None:
            print(feat['properties']['stop_name'])
                
def get_geo_dict(geo_json_map, geo_json_ratp):
    map_list_dict = []
    for item in geo_json_map['features']:
        for feat in geo_json_ratp['features']:
            if strip_accents(item["properties"]["Name"]) in strip_accents(feat['properties']['stop_name']):
                map_list_dict.append({'stop_name' : feat['properties']['stop_name'], 'stop_id': feat['properties']['stop_id'],
                                 'line': item["properties"]["Ligne"]})
    return map_list_dict

def insert_ratp_dict(geo_json_map, geo_ratp):
    map_list_dict = []
    for item, feat in zip(geo_json_map,geo_ratp):
        if item['geometry']['coordinates'] == feat['geometry']['coordinates']:
            map_list_dict.append({"type":"Feature",
                                  "properties":{'stop_id': item['properties']['stop_id'],
                                                'stop_name' : feat['properties']['stop_name'], 
                                                'stop_line': item['properties']['stop_line'],
                                                'stop_info': item["properties"]["stop_info"],
                                                'open_date': lambda x: if isinstance(feat['properties']['open_date'], datetime.datetime) feat["properties"]["open_date"].strftime('%Y-%m-%d'),
                                                'close_date': feat["properties"]["close_date"].strftime('%Y-%m-%d'),
                                                'start_map': item["properties"]["start_map"],
                                                'end_map': item["properties"]["end_map"],
                                                'ratp_id': feat['properties']['stop_id']},
                                  "geometry":item["geometry"]})
    return map_list_dict    

def get_df(list_dict):
    df = pd.DataFrame(list_dict)
    df.drop_duplicates('stop_name',inplace=True)
    df = df.reset_index()
    df = df.drop('index', axis=1)
    return df

def check_dup(geo, df):
    dupe = False
    for stop in df.stop_name.values:
        if strip_accents(geo) == strip_accents(stop):
            dupe = True      
    return dupe

def get_delta(geo_map, df):
    delta = []
    for feat in geo_map['features']:
        if not check_dup(feat["properties"]["Name"],df):
            delta.append({"Name": feat["properties"]["Name"], "line": feat["properties"]["Ligne"]})
    return delta

def remove_items(delta, stations_list):
    for station in stations_list:
        for item in delta:
            if item["Name"] == station:
                item.pop("Name")
                item.pop("line")
    while {} in delta:
        delta.remove({})
    return delta

def fill_delta(geo_ratp, delta_list_dict, station, line_num):
    for feat in geo_ratp['features']:
        if strip_accents(station) in strip_accents(feat['properties']['stop_name']):
            delta_list_dict.append({'stop_name' : feat['properties']['stop_name'], 'stop_id': feat['properties']['stop_id'],
                                   'line': line_num})
    return delta_list_dict

def remove_station(df, station):
    df = df[df.stop_name != station]
    return df

def remove_index(df, index):
    df = df.drop(df.index[index])
    return df

def merge_df(df1,df2):
    frames = [df1,df2]
    df = pd.concat(frames).reset_index()
    df = df.drop('index', axis=1)
    return df

def merge_df_three(df1,df2,df3):
    frames = [df1,df2,df3]
    df = pd.concat(frames).reset_index()
    df = df.drop('index', axis=1)
    return df

def add_date(df, station, date):
    df.loc[df["stop_name"] == station, ["open_date"]] = date
    return df

def add_closedate(df, station, date):
    df.loc[df["stop_name"] == station, ["close_date"]] = date
    return df

def add_station(df, line, id_, name, add, close):
    df.loc[-1] = [line,id_, name , add, close]
    df.index = df.index + 1  # shifting index
    df = df.sort_index()  # sorting by index
    return df

def make_excel(df, name):
    writer = pd.ExcelWriter('excels/'+name+'.xlsx')
    df.to_excel(writer,'Sheet1')
    writer.save()
    
def random_num():
    return random.randint(1111,9999)

In [66]:
geo_paris_dict = insert_ratp_dict(geo_database, check_geo)

AttributeError: 'str' object has no attribute 'strftime'

In [62]:
len(geo_paris_dict)

425

In [63]:
len(geo_database)

425

In [35]:
test_dict = []
test_dict.append({'type':'Feature','properties':{"Name": 'Yev', 'Last': 'Chervonenko'},'geometry':{'type':'Point','coord':'12345'}})
test_dict.append({'type':'Feature','properties':{"Name": 'Kevin', 'Last': 'Blin'},'geometry':{'type':'Point','coord':'12345'}})

In [36]:
test_dict

[{'type': 'Feature',
  'properties': {'Name': 'Yev', 'Last': 'Chervonenko'},
  'geometry': {'type': 'Point', 'coord': '12345'}},
 {'type': 'Feature',
  'properties': {'Name': 'Kevin', 'Last': 'Blin'},
  'geometry': {'type': 'Point', 'coord': '12345'}}]

In [22]:
delta = get_delta(paris_geo_1915, paris_1915_df)

In [ ]:
delta_list_dict = []

In [ ]:
paris_1915_delta = pd.DataFrame(delta_list_dict)
paris_1915_delta.drop_duplicates('stop_name',inplace=True)
paris_1915_delta = paris_1915_delta.reset_index()
paris_1915_delta = paris_1915_delta.drop('index', axis=1)
paris_1915_delta

In [ ]:
paris_1915_delta = paris_1915_delta.drop(paris_1915_delta.index[[0]])
paris_1915_delta

In [ ]:
paris_1915_delta = paris_1915_delta.drop(paris_1915_delta.index[[4]])

In [ ]:
paris_1915_delta

In [ ]:
frames = [paris_1915_delta,paris_1915_df]
frames

In [ ]:
paris_1915 = pd.concat(frames).reset_index()
paris_1915 = paris_1915.drop('index', axis=1)
paris_1915

In [ ]:
paris_1915['open_date'] = ''

In [ ]:
paris_1915 = paris_1915[paris_1915.stop_name != ""]

In [ ]:
paris_1915.loc[paris_1915["stop_name"] == "Eglise d'Auteuil", ["stop_name"]] = "Wilhem"


In [ ]:
paris_1915.loc[-1] = ["8",random.randint(1111,9999),"Champ de Mars",""]
paris_1915.index = paris_1915.index + 1  # shifting index
paris_1915 = paris_1915.sort_index()  # sorting by index

In [ ]:
paris_1915

In [ ]:
paris_1915.loc[paris_1915.line == "3"]

In [ ]:
paris_1915.loc[paris_1915["stop_name"] == "Porte de Champerret", ["open_date"]] = '1911-02-15'

In [ ]:
paris_1915

In [ ]:
writer = pd.ExcelWriter('paris_1915.xlsx')
paris_1915.to_excel(writer,'Sheet1')
writer.save()

In [37]:
list_dict_1950 = get_geo_dict(paris_geo_1950,geo_json_ratp)

In [38]:
paris_1950_df = get_df(list_dict_1950)

In [39]:
delta_1950 = get_delta(paris_geo_1950, paris_1950_df)

In [184]:
delta_list_1950 = []

In [229]:
delta_catch_1950 = fill_delta(geo_json_ratp,delta_list_1950, "Sablons", "1")

In [231]:
paris_1950_delta_df = get_df(delta_catch_1950)

In [238]:
paris_1950 = merge_df(paris_1950_df,paris_1950_delta_df)

In [453]:
paris_1908 = remove_station(paris_1908,"Martin Nadaud")

In [419]:
paris_1908 = pd.read_excel('excels/paris_1908.xlsx')

In [383]:
paris_1915['close_date'] = ''

In [475]:
#paris_1908.loc[paris_1908.stop_name == "Châtelet",["stop_id"]] = 1966
paris_1908.loc[paris_1908.stop_name == "Gare d'Austerlitz"]
               

,stop_name,stop_id,coord,line,open_date,close_date
113,Gare d'Austerlitz,1951,"[48.84341422414371, 2.3641883928676424]",NaN,1930-10-15,NaN


In [474]:
paris_1908.loc[paris_1908.stop_name == "Gare d'Orléans"]

,stop_name,stop_id,coord,line,open_date,close_date
112,Gare d'Orléans,1951,"[48.84371986136062, 2.3639162171627124]",NaN,1906-06-02,1930-10-15


In [407]:
paris_1915 = add_station(paris_1915, 8, 2057, "Avenue Emile-Zola", '1913-07-13','')


In [434]:
paris_1908 = add_date(paris_1908, "Jaurès", '1914-08-01')

In [473]:
paris_1908 = add_closedate(paris_1908, "Gare d'Orléans", '1930-10-15')

In [459]:
#paris_1908 = add_station(paris_1908, 8, 2031, "Charles Michels", '1945-07-14','')
paris_1908.loc[-1] = ["Arsenal",random_num(), '' , '', '1906-12-17', '1939-09-02']
paris_1908.index = paris_1908.index + 1  # shifting index
paris_1908 = paris_1908.sort_index()  # sorting by index


In [468]:
search_station("Gare d'Austerlitz")

stop_name : Gare d'Austerlitz  |stop_id : 1952
stop_name : Gare d'Austerlitz  |stop_id : 2206
stop_name : Gare d'Austerlitz  |stop_id : 1951
stop_name : Gare d'Austerlitz  |stop_id : 2207


In [477]:
make_excel(paris_1908, 'paris_1908')

In [952]:
paris_1908 = pd.read_excel('excels/paris_1908.xlsx')
paris_1915 = pd.read_excel('excels/paris_1915.xlsx')
paris_1950 = pd.read_excel('excels/paris_1950.xlsx')

In [953]:
metro_paris = merge_df_three(paris_1908,paris_1915,paris_1950)

/Users/yev/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:87: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [14]:
def rename_station_geo(geo):
    for item in metro_paris.values:
        for feat in geo:
            if strip_accents(feat['properties']["stop_name"]) == strip_accents(item[5]):
                feat['properties']["stop_name"] = item[5]
                feat['properties']["open_date"] = item[3]
                feat['properties']["stop_id"] = item[4]
                feat['properties']["close_date"] = item[0]
            
def rename_station(from_, to_):
    for feat in geo_database:
        if strip_accents(feat['properties']["stop_name"]) == strip_accents(from_):
            feat['properties']["stop_name"] = to_

def save_geo(geo, name):
    #Write result to a new file
    with open('Database/'+name+'.geojson', 'w') as f:
        json.dump(geo, f)

def manual_adjust(geo, geo_name, df_name):
    array = metro_paris.loc[metro_paris.stop_name == df_name].values[0]
    for feat in geo:
        if strip_accents(feat['properties']["stop_name"]) == strip_accents(geo_name):
            feat['properties']["open_date"] = array[3]
            feat['properties']["stop_id"] = array[4]
            feat['properties']["close_date"] = array[0]

In [942]:
metro_paris.loc[metro_paris.stop_name == "Etoile"].values[0][4]

2030

In [787]:
rename_station_geo()

In [786]:
geo_tmp = geo_database.copy()

In [975]:
search_empty(geo_temp)

In [982]:
rename = search_station_df(metro_paris,"grenelle")

['Bir-Hakeim (Grenelle)', 'La Motte-Picquet-Grenelle', 'Beaugrenelle']


In [974]:
manual_adjust(geo_temp, "Gare d'Orleans - Austerlitz", rename[0])

In [672]:
rename_station("La Motte-Pichet Grenelle",rename[0])

In [978]:
for item in geo_temp:
    if isinstance(item['properties']['close_date'], datetime.datetime):
        item['properties']['close_date'] = item['properties']['close_date'].strftime('%Y/%m/%d')

In [64]:
save_geo(geo_paris_dict,'paris_metropolitan_evolution_db_updated')

In [67]:
check_geo = json.load(open('Database/paris_metropolitan_evolution_db_updated.geojson'))

In [68]:
for item in check_geo:
    print(item['properties'])

{'stop_id': 19081, 'stop_name': 'Porte Maillot', 'stop_line': '1', 'stop_info': None, 'open_date': '1900-07-19', 'close_date': nan, 'start_map': 1908, 'end_map': 1950, 'ratp_id': 2433}
{'stop_id': 19082, 'stop_name': 'Porte Dauphine', 'stop_line': '2 Nord', 'stop_info': None, 'open_date': '1900-12-13', 'close_date': nan, 'start_map': 1908, 'end_map': 1908, 'ratp_id': 2414}
{'stop_id': 19083, 'stop_name': 'Porte Dauphine', 'stop_line': '2 Nord', 'stop_info': None, 'open_date': '1900-12-13', 'close_date': nan, 'start_map': 1908, 'end_map': 1908, 'ratp_id': 2414}
{'stop_id': 19084, 'stop_name': 'Victor Hugo', 'stop_line': '2 Nord', 'stop_info': None, 'open_date': '1900-12-13', 'close_date': nan, 'start_map': 1908, 'end_map': 1908, 'ratp_id': 2504}
{'stop_id': 19085, 'stop_name': 'Obligado', 'stop_line': '1', 'stop_info': None, 'open_date': '1900-07-19', 'close_date': '1948-05-25', 'start_map': 1908, 'end_map': 1915, 'ratp_id': 2048}
{'stop_id': 19086, 'stop_name': 'Etoile', 'stop_line': '

In [21]:
for item in geo_database:
    print(item['geometry']['coordinates'])

[2.283875687160487, 48.87671727241535]
[2.277593685385499, 48.87225307885228]
[2.277465602593661, 48.87078012674614]
[2.28559885987542, 48.86994758859919]
[2.288288598504033, 48.87532706585641]
[2.295141027867405, 48.87475069329314]
[2.298535221851131, 48.878080845880945]
[2.303274285149164, 48.879169549611575]
[2.308141431239036, 48.880514418925884]
[2.313072618724827, 48.881218874281]
[2.292195123655115, 48.87058800255838]
[2.300072215353196, 48.87142054070533]
[2.289761550610179, 48.86725784997057]
[2.319412716920844, 48.88217949521979]
[2.304939361443068, 48.870011629995105]
[2.287648184544839, 48.86418386296644]
[2.284766321728468, 48.85867630291738]
[2.313456867100343, 48.86738593276241]
[2.327738098390362, 48.88397265430553]
[2.331772706333282, 48.88397265430553]
[2.337408349174186, 48.88269182638714]
[2.32287095230049, 48.87596747981561]
[2.288993053859147, 48.854001281015265]
[2.324471987198474, 48.86578489786443]
[2.329915505851621, 48.864119821570526]
[2.336767935214993, 48.